In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 1
setup_one_gpu(GPU)

Picking GPU 1


In [2]:
import sys
import numpy as np
import os.path as osp
from tf_lab.iclr.evaluator import Evaluator
from general_tools.in_out.basics import files_in_subdirs
from tf_lab.iclr.helper import stored_synthetic_samples, top_evaluation_dir

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
class_name = 'chair'
do_mmd = True

save_res = True

top_out_dir = top_evaluation_dir

sample_mmd = False

n_samples = 1

ref_pop = 100

sample_pop = 500

mmd_loss = 'emd'

special_tag = 'test_emd_mmd_all_gt_boost_3'

skip = ['train', 'test']

boost_samples = 3

In [5]:
evaluator = Evaluator(class_name)
evaluator.load_gt_data()

In [6]:
best_models = {'gmm': ['bneck_128_full_32_gaussians', 
                       'bneck_128_full_34_gaussians',
                       'bneck_128_full_38_gaussians',
                       'bneck_128_full_40_gaussians'
                      ],
               'l_w_gan_large': [1700, 1800, 1900, 2000],
               'r_gan': [1700, 1800, 1900, 2000],
               'l_gan_chamfer': [100,200,300,400],
               'l_gan_emd': [100,200,300,400],
               'l_w_gan_small': [1700, 1800, 1900, 2000]
              }

In [7]:
generators_zoo = stored_synthetic_samples(class_name).keys()

In [9]:
epochs_to_eval = [1, 10, 100, 200, 400, 1000, 1500, 2000]
for experiment_name in generators_zoo:
    if 'gmm' in experiment_name: 
        continue

    print experiment_name
    
    top_sample_dir = stored_synthetic_samples(class_name)[experiment_name]
    if save_res:
        exp_name = osp.basename(top_sample_dir)
        f_mmd = open(osp.join(top_out_dir, class_name, 'mmd_' + special_tag + '_' + exp_name + '.txt'), 'w')
    else:            
        f_mmd = sys.stdout

    sample_files = []

    for epoch in epochs_to_eval:
        sample_files.append(osp.join(top_sample_dir, 'epoch_%d.npz' % (epoch,) ))
    
    for sample_file in sample_files:
        evaluator.prepare_sample_data(sample_file, boost_samples)
        sample_name = osp.basename(sample_file)[:-len('.npz')]
        print sample_name

        f_mmd.write(sample_name + '\n')

        evaluator.compute_mmd(loss=mmd_loss, sample_estimator=sample_mmd, n_samples=n_samples, 
                          ref_pop_size=ref_pop, sample_pop_size=sample_pop,
                          f_out=f_mmd, skip=skip)
        
        f_mmd.flush()
    print
    if save_res:
        f_mmd.close()

l_w_gan_large
epoch_10
val 185.706 56.6985

r_gan
epoch_10
val 299.532 41.2538

l_gan_chamfer
epoch_10
val 193.62 57.4797

l_w_gan_small
epoch_10
val 203.185 56.2191

l_gan_emd
epoch_10
val 183.866 55.0388



In [8]:
for experiment_name in generators_zoo:
    print experiment_name
    top_sample_dir = stored_synthetic_samples(class_name)[experiment_name]
    if save_res:
        exp_name = osp.basename(top_sample_dir)
        f_mmd = open(osp.join(top_out_dir, class_name, 'mmd_' + special_tag + exp_name + '.txt'), 'w')
    else:            
        f_mmd = sys.stdout

    sample_files = []

    if 'gan' in experiment_name:
        epochs = best_models[experiment_name]
        for epoch in epochs:
            sample_files.append(osp.join(top_sample_dir, 'epoch_%d.npz' % (epoch,) ))
    elif 'gmm' in experiment_name:
        gaussian_types = best_models[experiment_name]
        for gaussian_type in gaussian_types:
            sample_files.append(osp.join(top_sample_dir, gaussian_type + '.npz'))
    else:
        assert False
    
    for sample_file in sample_files:
        evaluator.prepare_sample_data(sample_file, boost_samples)
        sample_name = osp.basename(sample_file)[:-len('.npz')]
        print sample_name

        f_mmd.write(sample_name + '\n')
        evaluator.compute_mmd(loss=mmd_loss, sample_estimator=sample_mmd, n_samples=n_samples, 
                          ref_pop_size=ref_pop, sample_pop_size=sample_pop,
                          f_out=f_mmd, skip=skip)

    f_mmd.flush()
    print
    if save_res:
        f_mmd.close()

l_w_gan_large
epoch_1700
train 134.518 2.3447
val 137.519 2.33123
epoch_1800
train 133.695 2.3081
val 136.843 1.73321
epoch_1900
train 138.437 1.88252
val 134.845 1.04893
epoch_2000
train 134.465 0.220914
val 136.067 1.74778

gmm
bneck_128_full_32_gaussians
train 133.131 1.90576
val 137.997 1.67329
bneck_128_full_34_gaussians
train 134.338 0.238103
val 138.717 0.59688
bneck_128_full_38_gaussians
train 132.457 1.62623
val 133.727 3.6899
bneck_128_full_40_gaussians
train 134.838 2.77159
val 137.205 1.36068

r_gan
epoch_1700
train 253.007 1.64227
val 250.81 1.63932
epoch_1800
train 252.875 1.17974
val 251.764 0.927074
epoch_1900
train 255.411 3.36482
val 252.374 1.69515
epoch_2000
train 247.04 1.94329
val 247.384 2.9045

l_gan_chamfer
epoch_100
train 167.463 6.88921
val 172.177 6.11821
epoch_200
train 167.761 3.15888
val 168.974 2.15308
epoch_300
train 168.357 3.48345
val 163.465 2.17416
epoch_400
train 167.564 4.42508
val 167.005 2.08013

l_w_gan_small
epoch_1700
train 137.818 3.54374
va